<a href="https://colab.research.google.com/github/bankijey/IBM-DS-Capstone-Project/blob/main/Weather_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Collecting Weather Data Using APIs**


The API at [OpenWeather](https://openweathermap.org/current#name) gives us the information of Current weather.<br>

In [1]:
%load_ext google.colab.data_table
import numpy as np 
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import io
# Geographical libraries
try:
    import geopy
except:
    !conda install -c conda-forge geopy --yes
    import geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=10)


geopy.geocoders.options.default_user_agent = 'my_app/banki'
geopy.geocoders.options.default_timeout = 7
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance

try:
  import geopandas as geopd
except:
  !pip install geopandas
  import geopandas as geopd

from shapely.geometry import Point, Polygon

# Web Scraping
import requests # library to handle requests
from bs4 import BeautifulSoup # this module helps in web scrapping.
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import folium # map rendering library
from folium.plugins import MarkerCluster
from folium import IFrame

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import seaborn as sns # for statistical analysis
import missingno as msno
%matplotlib inline

# Import Clustering algorithms
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

print('Package load: Successful')

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


     |████████████████████████████████| 1.0MB 8.8MB/s 
     |████████████████████████████████| 15.3MB 272kB/s 
     |████████████████████████████████| 6.5MB 36.7MB/s 
Package load: Successful


#### Country City list data

In [ ]:
city_list = pd.read_json('city.list.json')
# city_list.country.value_counts()

In [ ]:
SA = list(city_list.name[(city_list.country=='ZA')])
# Need SQL Schema !!!
SA

555

In [ ]:
def weather_data(city_name):

    api_url = "http://api.openweathermap.org/data/2.5/weather?"
    payload= {'q': city_name, 'appid': "40917f9a7c56d52af84a55657cd0caa1"}
    response = requests.get(api_url, params = payload ) # Call the API using the get method and store the
    if response.ok:             # if all is well() no errors, no network timeouts)
        data = response.json()
    
    
    weather = data.get('main')
    temp = weather.get('temp')
    temp_min = weather.get('temp_min')
    temp_max = weather.get('temp_max')
    pressure = weather.get('pressure')
    humidity = weather.get('humidity')

    return temp, pressure, humidity



In [ ]:
#city_name = SA[500]
city_name = input()
temp, pressure, humidity = weather_data(city_name)
print( 'The current temperature in {0} is {1}°C. The humidity is {2}% and pressure is {3} kPa'.format(city_name,
                                                                                              temp-273.15, 
                                                                                              humidity, pressure))

Alice, ZA
The current temperature in Alice, ZA is 20.980000000000018°C. The humidity is 58% and pressure is 1019 kPa


Anthony Jeyifous

In [2]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_municipalities_in_South_Africa')
cities_df = tables[2]
cities_df.drop_duplicates('Seat', inplace=True)
cities_df
weather_df = pd.DataFrame(columns=['City', 'Temperature', 'Pressure', 'Humidity'])

for city in cities_df['Seat']:
  api_url = "http://api.openweathermap.org/data/2.5/weather?"
  payload= {'q': city + ', ZA', 'appid': "40917f9a7c56d52af84a55657cd0caa1"}
  response = requests.get(api_url, params = payload ) # Call the API using the get method and store the
  if response.ok:             # if all is well() no errors, no network timeouts)
    data = response.json()
    temp = round(data.get('main').get('temp') - 273.15,2)
    pres = data.get('main').get('pressure')
    hum = data.get('main').get('humidity')
    weather_df.loc[len(weather_df)] = [city,temp,pres,hum]
  else:
    None

Add geo coordinates for maps

In [3]:
weather_df['full_address'] = weather_df.City + ', South Africa'
weather_df['gcode'] = weather_df.full_address.apply(geolocator.geocode)
lats = []
longs = []
for idx in range(weather_df.shape[0]):
  if weather_df.loc[idx,'gcode'] != None:
    lat = weather_df.loc[idx,'gcode'].latitude
    longi = weather_df.loc[idx,'gcode'].longitude
  else:
    lat = None
    longi = None
  lats.append(lat)
  longs.append(longi)
weather_df['Latitude'] = lats
weather_df['Longitude'] = longs
weather_df.drop(columns= 'full_address', inplace= True)
# weather_df.dropna(axis=0, inplace=True)
weather_df

,City,Temperature,Pressure,Humidity,gcode,Latitude,Longitude
0,Newcastle,28.87,1017,24,"(Newcastle, Newcastle Local Municipality, Amaj...",-27.762778,29.923889
1,East London,21.00,1016,83,"(East London, Buffalo City Metropolitan Munici...",-33.019160,27.899857
2,Rustenburg,29.17,1017,21,"(Rustenburg, Rustenburg Local Municipality, Bo...",-25.665655,27.241448
3,Worcester,26.97,1015,29,"(Worcester, Breede Valley Local Municipality, ...",-33.646238,19.445927
4,Polokwane,24.00,1024,53,"(Polokwane, Polokwane Local Municipality, Capr...",-23.905833,29.461389
5,Beaufort West,30.35,1014,16,"(Beaufort West, Beaufort West Local Municipali...",-32.353611,22.580556
6,Queenstown,31.76,1013,16,"(Queenstown, Livingstone Road, The Hexagon, Lu...",-31.892843,26.877681
7,Cape Town,23.02,1015,68,"(Cape Town, City of Cape Town, Western Cape, 8...",-33.928992,18.417396
8,Germiston,25.65,1026,31,"(Germiston, City of Ekurhuleni Metropolitan Mu...",-26.229444,28.180556
9,Johannesburg,26.13,1026,31,"(Johannesburg, City of Johannesburg Metropolit...",-26.205000,28.049722


In [ ]:
# Create map with the neighbourhood coordinates
location = geolocator.geocode('South Africa')
lati = location.latitude
longi = location.longitude
map_CT = folium.Map(location=[lati, longi], zoom_start=6, min_zoom= 4, max_zoom= 10)
for lat, lng, city in zip(weather_df['Latitude'], weather_df['Longitude'], weather_df['City']):
        label = city
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=3,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_CT)  

map_CT

## 2.0 Scrape data for List of south african provinces

South Africa has 13 provinces. Each province has unique demographic data like population, human development index and density and these are scraped from the web along with the geocordinates and stored in a database in a database. 

#### 2,1 Scrape data for list of south african provinces

Data of South african provinces are scraped from [Wikipedia](https://en.wikipedia.org/wiki/Provinces_of_South_Africa)

In [ ]:
url = 'https://en.wikipedia.org/wiki/Provinces_of_South_Africa'  
tables = pd.read_html(url)

df = tables[2]
df
# df.dtypes

,Province,Name in the most spoken native language[9],Capital,Largest city,Area[10]:9,Population(2011)[10]:18,Population density(2011),Human Devel.Index (2003) [11],Location
0,Eastern Cape,iMpuma-Koloni (Xhosa),Bhisho (Bisho),Gqeberha,"168,966 km2 (65,238 sq mi)",6562053,38.8/km2 (100/sq mi),0.62,NaN
1,Free State,Freistata (Sotho),Bloemfontein,Bloemfontein,"129,825 km2 (50,126 sq mi)",2745590,21.1/km2 (55/sq mi),0.67,NaN
2,Gauteng,eGoli (Zulu),Johannesburg,Johannesburg,"18,178 km2 (7,019 sq mi)",12272263,"675.1/km2 (1,749/sq mi)",0.74,NaN
3,KwaZulu-Natal,iKwaZulu-Natali (Zulu),Pietermaritzburg [n 1],Durban,"94,361 km2 (36,433 sq mi)",10267300,108.8/km2 (282/sq mi),0.63,NaN
4,Limpopo,Limpopo (Northern Sotho),Polokwane (Pietersburg),Polokwane,"125,754 km2 (48,554 sq mi)",5404868,43.0/km2 (111/sq mi),0.59,NaN
5,Mpumalanga,iMpumalanga (Swazi),Mbombela (Nelspruit),Mbombela,"76,495 km2 (29,535 sq mi)",4039939,52.8/km2 (137/sq mi),0.65,NaN
6,North West,Bokone Bophirima (Tswana),Mahikeng (Mafikeng),Klerksdorp,"104,882 km2 (40,495 sq mi)",3509953,33.5/km2 (87/sq mi),0.61,NaN
7,Northern Cape,Noord-Kaap (Afrikaans),Kimberley,Kimberley,"372,889 km2 (143,973 sq mi)",1145861,3.1/km2 (8.0/sq mi),0.69,NaN
8,Western Cape [n 2],Wes-Kaap (Afrikaans),Cape Town,Cape Town,"129,462 km2 (49,986 sq mi)",5822734,45.0/km2 (117/sq mi),0.77,NaN
9,Republic of South Africa,iRiphabhuliki yaseNingizimu Afrika (Zulu),"Pretoria, Cape Town, Bloemfontein[citation nee...",Johannesburg,"1,220,813 km2 (471,359 sq mi)",51770560,42.4/km2 (110/sq mi),0.67,NaN


Edit dataframe columns 

In [ ]:
clmns = list(df.columns) # List of required columns
new_clmns = ['Province', 
             'Top_Language',
             'Capital',
             'Largest_City',
             'Area',
             'Population_2011',
             'PopDens_2011',
            'HDI_2003']  # New names of columns

df_prov = df.loc[0:8,clmns].rename(columns = dict(zip(clmns, new_clmns)))

df_prov.loc[8,'Province'] = 'Western Cape'
df_prov.drop('Location', axis = 1, inplace = True)

df_prov = df_prov.convert_dtypes()
df_prov


,Province,Top_Language,Capital,Largest_City,Area,Population_2011,PopDens_2011,HDI_2003
0,Eastern Cape,iMpuma-Koloni (Xhosa),Bhisho (Bisho),Gqeberha,"168,966 km2 (65,238 sq mi)",6562053,38.8/km2 (100/sq mi),0.62
1,Free State,Freistata (Sotho),Bloemfontein,Bloemfontein,"129,825 km2 (50,126 sq mi)",2745590,21.1/km2 (55/sq mi),0.67
2,Gauteng,eGoli (Zulu),Johannesburg,Johannesburg,"18,178 km2 (7,019 sq mi)",12272263,"675.1/km2 (1,749/sq mi)",0.74
3,KwaZulu-Natal,iKwaZulu-Natali (Zulu),Pietermaritzburg [n 1],Durban,"94,361 km2 (36,433 sq mi)",10267300,108.8/km2 (282/sq mi),0.63
4,Limpopo,Limpopo (Northern Sotho),Polokwane (Pietersburg),Polokwane,"125,754 km2 (48,554 sq mi)",5404868,43.0/km2 (111/sq mi),0.59
5,Mpumalanga,iMpumalanga (Swazi),Mbombela (Nelspruit),Mbombela,"76,495 km2 (29,535 sq mi)",4039939,52.8/km2 (137/sq mi),0.65
6,North West,Bokone Bophirima (Tswana),Mahikeng (Mafikeng),Klerksdorp,"104,882 km2 (40,495 sq mi)",3509953,33.5/km2 (87/sq mi),0.61
7,Northern Cape,Noord-Kaap (Afrikaans),Kimberley,Kimberley,"372,889 km2 (143,973 sq mi)",1145861,3.1/km2 (8.0/sq mi),0.69
8,Western Cape,Wes-Kaap (Afrikaans),Cape Town,Cape Town,"129,462 km2 (49,986 sq mi)",5822734,45.0/km2 (117/sq mi),0.77


Create a list of provinces 

Loop through the list of provinces to get location coordinates and add to province dataframe

In [ ]:
#  Instantiate column
df_loc = pd.DataFrame(columns = ['Province', 'Latitude', 'Longitude'])

for prov in list(df_prov['Province']):
    address = prov + ', South Africa'
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)

    df_loc = df_loc.append({'Latitude': location.latitude,
                    'Longitude': location.longitude,
                           'Province': prov}, ignore_index=True)
#     print('The coordinates of {} is {} and {}'.format(address, longitude, latitude))
   

In [ ]:
df_loc

,Province,Latitude,Longitude
0,Eastern Cape,-32.217183,26.638640
1,Free State,-28.785362,26.497893
2,Gauteng,-25.936344,28.081311
3,KwaZulu-Natal,-28.503833,30.887501
4,Limpopo,-23.473529,29.395920
5,Mpumalanga,-26.276849,30.150015
6,North West,-26.134782,25.654673
7,Northern Cape,-29.573402,21.205136
8,Western Cape,-33.546977,20.727530


In [ ]:
add = 'South Africa'
lati, longi = getCoord(add)
print('Lat: {} and Long: {}'.format(lati, longi))


Lat: -28.579397200000002 and Long: 24.086794125366062


In [ ]:
maps = LoopMap(df = df_loc, add = 'South Africa', zoom_start = 6, label = 'Province')
maps